In [1]:
import pandas as pd
import folium
import random
import math
from collections import deque


In [2]:
# Load customer data
df = pd.read_csv(r'C:\Users\Samuel\Documents\customers_gwagwalada.csv')

# Extract locations and demands
depot = (df.iloc[0]['latitude'], df.iloc[0]['longitude'])
customers = df.iloc[1:].to_dict('records')
locations = [depot] + [(customer['latitude'], customer['longitude']) for customer in customers]
demands = [customer['demand_bags'] + customer['demand_packs'] for customer in customers]
print(df.head())

   id  latitude  longitude  demand_bags  demand_packs
0   0  8.942050   7.081990            0             0
1   1  8.925081   7.052678            8             3
2   2  8.900276   7.067616           16             3
3   3  8.903489   7.050503            3             4
4   4  8.940240   7.094881           14             1


In [3]:
# Calculate distance matrix (Haversine distance)
def haversine_distance(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

distance_matrix = [[haversine_distance(*loc1, *loc2) for loc2 in locations] for loc1 in locations]


In [4]:
def get_initial_solution(demands, capacity):
    routes = []
    remaining_customers = set(range(len(demands)))
    vehicle_load = 0
    current_route = []
    while remaining_customers:
        for customer in list(remaining_customers):
            if vehicle_load + demands[customer] <= capacity:
                current_route.append(customer)
                vehicle_load += demands[customer]
                remaining_customers.remove(customer)
                break
        else:  # No more customers can fit in the current vehicle
            routes.append(current_route)
            current_route = []
            vehicle_load = 0
    if current_route:  # Add the last route if it's not empty
        routes.append(current_route)
    return routes

# Parameters
num_vehicles = 5
capacity = 220
initial_solution = get_initial_solution(demands, capacity)

In [5]:
def swap_2opt(solution, i, j):
    new_solution = solution[:]
    new_solution[i:j+1] = reversed(new_solution[i:j+1])
    return new_solution

def relocate_customer(solution, route_idx1, route_idx2, customer_index):
    new_solution = [route[:] for route in solution]  # Deep copy of the solution
    customer = new_solution[route_idx1].pop(customer_index)
    new_solution[route_idx2].append(customer)
    return new_solution


In [6]:
def generate_neighborhood(solution, distance_matrix, tabu_list, best_solution):  
    neighborhood = []
    for route_idx, route in enumerate(solution):
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route)):
                neighbor = swap_2opt(solution, i, j)
                if neighbor not in tabu_list and total_distance(neighbor, distance_matrix) < total_distance(best_solution, distance_matrix):
                    neighborhood.append(neighbor)

    for route_idx1, route1 in enumerate(solution):
        for route_idx2, route2 in enumerate(solution):
            if route_idx1 != route_idx2:
                for i in range(1, len(route1)):
                    neighbor = relocate_customer(solution, route_idx1, route_idx2, i)
                    if neighbor not in tabu_list:
                        neighborhood.append(neighbor)
    return neighborhood

In [7]:
def total_distance(solution, distance_matrix):
    total = 0
    for route in solution:
        route = [0] + route + [0]  # Add depot at start and end
        for i in range(len(route) - 1):
            total += distance_matrix[route[i]][route[i+1]]
    return total

In [8]:
def tabu_search(initial_solution, distance_matrix, max_iterations=100, tabu_tenure=10):
    current_solution = initial_solution
    best_solution = initial_solution
    tabu_list = deque(maxlen=tabu_tenure)

    for _ in range(max_iterations):
        neighborhood = generate_neighborhood(current_solution, distance_matrix, tabu_list, best_solution)
        best_neighbor = None
        best_neighbor_cost = float('inf')

        for neighbor in neighborhood:
            neighbor_cost = total_distance(neighbor, distance_matrix)
            if neighbor not in tabu_list and neighbor_cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = neighbor_cost

            # Aspiration criterion (optional)
            elif neighbor in tabu_list and neighbor_cost < total_distance(best_solution, distance_matrix):
                best_neighbor = neighbor
                best_neighbor_cost = neighbor_cost

        if best_neighbor is not None:
            current_solution = best_neighbor
            if total_distance(current_solution, distance_matrix) < total_distance(best_solution, distance_matrix):
                best_solution = current_solution

            tabu_list.append(best_neighbor)

    return best_solution


In [9]:

# Run the Tabu Search
best_solution = tabu_search(initial_solution, distance_matrix)

# Print Best Solution
print("Best solution found:", best_solution)
print("Total distance in kilometers:", total_distance(best_solution, distance_matrix))

Best solution found: [[0, 1, 2, 3, 5, 6, 8, 11, 88, 94, 82, 63], [31, 32, 35, 36, 38, 42, 43, 44, 45, 72, 20, 28, 89, 62, 9, 80, 47, 86, 30, 33, 96], [15, 16, 17, 19, 21, 29, 77, 34], [95, 98, 99, 91, 37, 64, 51, 73, 71, 70, 41, 75, 93, 83, 25, 57, 26, 53, 27, 58, 84], [78, 81, 85, 90, 92, 97, 23, 13, 18, 60, 7, 54, 14, 24, 22, 4, 12, 87], [59, 61, 65, 66, 67, 68, 69, 74, 79, 10, 40, 39], [46, 48, 49, 50, 52, 55, 56, 76]]
Total distance in kilometers: 154.72656277142292


In [12]:
# Visualize with Folium

# Create Folium Map
map_center = depot
vrp_map = folium.Map(location=map_center, zoom_start=12, tiles="cartodb positron")

# Add Depot Marker
folium.Marker(
    location=depot,
    popup="Depot",
    icon=folium.Icon(color='red', icon='industry', prefix='fa'),
).add_to(vrp_map)

# Add Customer Markers with Popups
for idx, (lat, lon) in enumerate(locations[1:]):
    demand = demands[idx]
    folium.Marker(
        location=[lat, lon],
        popup=f"Customer {idx + 1}, Demand: {demand}",
    ).add_to(vrp_map)

colors = ['blue', 'green', 'purple', 'orange', 'red']  # Add more colors if needed
for vehicle_id, route in enumerate(best_solution):
    route_with_depot = [0] + route + [0]  # Add depot to start and end
    route_coordinates = [locations[i] for i in route_with_depot]
    folium.PolyLine(route_coordinates, color=colors[vehicle_id % len(colors)], weight=2.5, opacity=1).add_to(vrp_map)

# Show the map
vrp_map